In [67]:
import os
import itertools
import pandas as pd

Sns = [
    (100, 50),
    (100, 100),
    (100, 200),
    (200, 100),
    (200, 200),
    (200, 400),
]

disttypes = ["uniform", "half", "zipfhalf", "zipf", "diri", "dirihalf"]
altass_path_temp = "/Better-Turing/result/iclr/S{S}-n{n}-k0-{disttype}-nGT-max20/evolutions/seed{seed}/evo2/assess-incn.csv"

In [68]:
data_sampling = []

for Sn, disttype, smp_idx, evo_idx in itertools.product(
    Sns, disttypes, range(100), range(1)
):
    S, n = Sn
    altass_path = altass_path_temp.format(
        S=S, n=n, disttype=disttype, seed=smp_idx
    )
    if os.path.exists(altass_path):
        altass_df = pd.read_csv(altass_path)
        altass_df["S"] = S
        altass_df["n"] = n
        altass_df["disttype"] = disttype
        altass_df["seed"] = smp_idx
        data_sampling.append(altass_df)
    else:
        print(f"File not found: {altass_path}")
df_sampling = pd.concat(data_sampling)
display(df_sampling)

,c,mm,mm_gt,mm_evo,mse_gt,mse_evo,msre_gt,msre_evo,evo/gt,S,n,disttype,seed
0,2,0.373037,0.355382,0.360998,0.005239,0.003448,0.014045,0.009243,0.658115,100,50,uniform,0
1,5,0.082749,0.081370,0.086664,0.001131,0.001060,0.013674,0.012811,0.936955,100,50,uniform,0
2,10,0.006606,0.006689,0.007255,0.000078,0.000076,0.011883,0.011526,0.969973,100,50,uniform,0
0,2,0.372802,0.361414,0.419511,0.005259,0.004754,0.014106,0.012751,0.903959,100,50,uniform,1
1,5,0.082545,0.082241,0.106534,0.001132,0.001639,0.013717,0.019853,1.447374,100,50,uniform,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,5,0.009264,0.009891,0.009654,0.000012,0.000011,0.001282,0.001151,0.897799,200,400,dirihalf,98
2,10,0.003225,0.003418,0.003298,0.000002,0.000002,0.000608,0.000561,0.922297,200,400,dirihalf,98
0,2,0.035767,0.038725,0.036711,0.000094,0.000073,0.002626,0.002052,0.781509,200,400,dirihalf,99
1,5,0.009941,0.010559,0.009900,0.000012,0.000011,0.001250,0.001106,0.885107,200,400,dirihalf,99


In [69]:
df_sampling["evo/gt_mse"] = df_sampling["mse_evo"] / df_sampling["mse_gt"]

In [70]:
# per S, n, disttype, c, get statistics of msre_gt msre_evo, evo/gt
df_stat = df_sampling.groupby(["disttype", "S", "n", "c"]).agg(
    {
        "evo/gt_mse": ["mean", "median", "std"],
    }
)
# add wilcox signed rank test between mse_gt and mse_evo
from scipy.stats import wilcoxon
import numpy as np

from cliffs_delta import cliffs_delta
import numpy as np

df_stat["wilcoxon"] = np.nan
for idx, row in df_stat.iterrows():
    disttype, S, n, c = idx
    df = df_sampling[
        (df_sampling["disttype"] == disttype)
        & (df_sampling["S"] == S)
        & (df_sampling["n"] == n)
        & (df_sampling["c"] == c)
    ]
    _, p = wilcoxon(df["mse_gt"], df["mse_evo"], alternative="greater")
    df_stat.loc[idx, "wilcoxon"] = p
    # if p < 0.05: significant = True else False
    df_stat.loc[idx, "p<0.05"] = p < 0.05

pd.set_option("display.max_rows", None)
display(df_stat)
pd.reset_option("display.max_rows")

/tmp/ipykernel_3381159/3412367868.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_stat.loc[idx, "p<0.05"] = p < 0.05
/tmp/ipykernel_3381159/3412367868.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'small' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_stat.loc[idx, "vda_interpret"] = (


evo/gt_mse                          wilcoxon p<0.05  \
                          mean    median       std                        
disttype S   n   c                                                        
diri     100 50  2    0.986705  0.970953  0.256244  1.793009e-01  False   
                 5    0.963104  0.958746  0.076709  3.144364e-07   True   
                 10   0.961895  0.962637  0.040657  2.656493e-13   True   
             100 2    0.914803  0.911695  0.138533  5.728368e-09   True   
                 5    0.940231  0.932905  0.041221  6.941954e-17   True   
                 10   0.953773  0.952198  0.022279  1.948280e-18   True   
             200 2    0.936103  0.928097  0.103801  4.972182e-07   True   
                 5    0.942348  0.938822  0.034117  2.069687e-18   True   
                 10   0.951343  0.952382  0.020418  1.948280e-18   True   
         200 100 2    0.995697  0.961074  0.318792  2.090088e-02   True   
                 5    0.971289  0.969388  0.077414  9.144428e-08   True   
                 10   0.970437  0.969474  0.027256  7.076313e-15   True   
             200 2    1.016539  0.987930  0.227949  8.033279e-02  False   
                 5    0.974177  0.979816  0.039615  3.232198e-10   True   
                 10   0.974378  0.976353  0.018432  1.948280e-18   True   
             400 2    0.959031  0.947160  0.074649  2.245424e-08   True   
                 5    0.969831  0.972820  0.025328  1.202331e-15   True   
                 10   0.976382  0.977460  0.015499  1.948280e-18   True   
dirihalf 100 50  2    0.997637  0.931995  0.273553  4.004990e-02   True   
                 5    0.968960  0.944740  0.117004  1.120318e-05   True   
                 10   0.963121  0.954996  0.060337  3.816115e-09   True   
             100 2    0.946679  0.925359  0.161394  7.674901e-03   True   
                 5    0.952803  0.946135  0.086637  1.120682e-07   True   
                 10   0.955782  0.961868  0.050377  2.291783e-12   True   
             200 2    0.914271  0.915296  0.112312  1.815686e-09   True   
                 5    0.936204  0.935895  0.051508  2.117636e-16   True   
                 10   0.946363  0.952323  0.031373  1.948280e-18   True   
         200 100 2    0.956891  0.952019  0.169187  7.094821e-04   True   
                 5    0.965282  0.971001  0.059957  7.901662e-09   True   
                 10   0.975295  0.973759  0.032328  9.181072e-12   True   
             200 2    0.966569  0.979231  0.128839  1.716675e-03   True   
                 5    0.963604  0.967545  0.042883  7.077602e-12   True   
                 10   0.972067  0.975945  0.025039  1.270657e-15   True   
             400 2    0.947781  0.956958  0.106372  1.845718e-05   True   
                 5    0.967373  0.973485  0.044409  1.705557e-09   True   
                 10   0.972858  0.975597  0.026584  1.673964e-15   True   
half     100 50  2    1.099115  0.979906  0.402394  9.144149e-01  False   
                 5    1.003610  0.984987  0.101709  1.225865e-01  False   
                 10   0.975814  0.975900  0.024920  2.462388e-13   True   
             100 2    0.990539  0.949090  0.243068  2.746198e-04   True   
                 5    0.962621  0.962210  0.030921  8.864183e-16   True   
                 10   0.978447  0.977681  0.010811  2.008078e-18   True   
             200 2    0.966324  0.962817  0.091133  3.258073e-07   True   
                 5    0.977167  0.973852  0.012216  2.634602e-18   True   
                 10   0.988339  0.988076  0.005465  1.948280e-18   True   
         200 100 2    1.164896  0.988277  0.546695  8.583100e-01  False   
                 5    1.058499  1.000523  0.141440  9.926110e-01  False   
                 10   0.999971  0.997585  0.026639  1.225865e-01  False   
             200 2    1.124941  0.994775  0.382397  7.487220e-01  False   
                 5    1.006723  0.996030  0.061198  5.726086e-02  False   
                 10   0.993219  0.994127  0.0

In [71]:
df_stat = df_stat.reorder_levels(["S", "n", "disttype", "c"])
df_stat_sub = df_stat.loc[100, 100]
df_stat_sub = df_stat_sub[
    [
        ("evo/gt_mse", "median"), 
        ("p<0.05", ""), 
    ]
]
# drop column level
df_stat_sub.columns = df_stat_sub.columns.droplevel(1)
df_stat_sub = df_stat_sub.rename(columns={"evo/gt_mse": "Ratio"})
df_stat_sub = df_stat_sub.unstack("c").swaplevel(axis=1).sort_index(axis=1)
df_stat_sub = df_stat_sub.reindex(
    ["uniform", "half", "zipfhalf", "zipf", "diri", "dirihalf"]
)
df_stat_sub

/tmp/ipykernel_3381159/2679700481.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  df_stat_sub = df_stat.loc[100, 100]


c               2                5                10       
             Ratio p<0.05     Ratio p<0.05     Ratio p<0.05
disttype                                                   
uniform   1.004866  False  0.998710  False  0.991721   True
half      0.949090   True  0.962210   True  0.977681   True
zipfhalf  0.971420   True  0.983240   True  0.985753   True
zipf      0.933532   True  0.946000   True  0.966799   True
diri      0.911695   True  0.932905   True  0.952198   True
dirihalf  0.925359   True  0.946135   True  0.961868   True

In [72]:
# to latex
print(df_stat_sub.to_latex(float_format="%.2f"))

\begin{tabular}{lrlrlrl}
\toprule
c & \multicolumn{2}{r}{2} & \multicolumn{2}{r}{5} & \multicolumn{2}{r}{10} \\
 & Ratio & p<0.05 & Ratio & p<0.05 & Ratio & p<0.05 \\
disttype &  &  &  &  &  &  \\
\midrule
uniform & 1.00 & False & 1.00 & False & 0.99 & True \\
half & 0.95 & True & 0.96 & True & 0.98 & True \\
zipfhalf & 0.97 & True & 0.98 & True & 0.99 & True \\
zipf & 0.93 & True & 0.95 & True & 0.97 & True \\
diri & 0.91 & True & 0.93 & True & 0.95 & True \\
dirihalf & 0.93 & True & 0.95 & True & 0.96 & True \\
\bottomrule
\end{tabular}

